# Homework Starter — Stage 10a: Linear Regression

Use this as a scaffold for gold price prediction modeling.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.model_selection import train_test_split
import scipy.stats as st

sns.set()
np.random.seed(7)

## 1) Gold Price Data
Gold price prediction factors with quadratic effects.

In [ ]:
n = 200
dates = pd.bdate_range(start="2024-02-01", periods=n)
inflation_rate = np.random.normal(3.2, 0.8, size=n)
dollar_index = np.random.normal(102, 4, size=n)
bond_yield = np.random.normal(4.5, 0.6, size=n)
volatility = np.random.normal(0.25, 0.08, size=n)

beta0, beta_inf, beta_dxy, beta_yield, beta_vol, beta_vol2 = 2400, 25, -8, -45, 180, 150
noise_scale = 15 + 5*np.abs(inflation_rate)
eps = np.random.normal(0, noise_scale)
gold_price = (
    beta0 + beta_inf*inflation_rate + beta_dxy*dollar_index + beta_yield*bond_yield + beta_vol*volatility
    + beta_vol2*(volatility**2) + eps
)

df = pd.DataFrame({
    'date': dates,
    'inflation_rate': inflation_rate,
    'dollar_index': dollar_index,
    'bond_yield': bond_yield,
    'volatility': volatility,
    'gold_price': gold_price
})
df.head()

## 2) Baseline model fit

In [ ]:
X = df[['inflation_rate','dollar_index','bond_yield','volatility']]
y = df['gold_price']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)
lr = LinearRegression().fit(X_train, y_train)
y_pred = lr.predict(X_test)
r2 = r2_score(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred, squared=False)
print(f'Baseline   R²={r2:.4f}  RMSE={rmse:.6f}')

In [ ]:
resid = y_test - y_pred
fitted = y_pred
plt.figure(); plt.scatter(fitted, resid); plt.axhline(0, ls='--'); plt.title('Residuals vs Fitted'); plt.show()
plt.figure(); plt.hist(resid, bins=20); plt.title('Residual Histogram'); plt.show()
plt.figure(); st.probplot(resid, dist='norm', plot=plt); plt.title('QQ Plot'); plt.show()

## 3) Add transformed feature

In [ ]:
df['volatility_sq'] = df['volatility']**2
X2 = df[['inflation_rate','dollar_index','bond_yield','volatility','volatility_sq']]
X2_train, X2_test = X2.iloc[:len(X_train)], X2.iloc[len(X_train):]
lr2 = LinearRegression().fit(X2_train, y_train)
y_pred2 = lr2.predict(X2_test)
r2_2 = r2_score(y_test, y_pred2)
rmse_2 = mean_squared_error(y_test, y_pred2, squared=False)
print(f'With x^2   R²={r2_2:.4f}  RMSE={rmse_2:.6f}')

## 4) Interpretation

- **Linearity:** Residuals vs fitted plot shows if relationship is linear. Random scatter around zero line means good fit.
- **Homoscedasticity:** Equal spread of residuals across fitted values. Funnel shape means problem.
- **Normality:** QQ plot and histogram check if residuals are normal. Straight line in QQ plot is good.
- **Independence:** Time series data may have correlation between residuals.
- **Which model do you trust:** Model with volatility squared term shows better R² and RMSE scores.
- **Next step:** Check for more missing variables or try different model types.